In [1]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/mingyue/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(f"Using {device} device")

Using cuda device


In [ ]:
%run dataProcessing.ipynb

In [ ]:
train_df, test_df = train_test_split(dfr, test_size=0.3, random_state=42)

In [ ]:
# ---------------------------Reference-------------------
# ----------- All implementation are based on the Pytorch tutorial by PyTorch official website: https://pytorch.org/tutorials/
# ----------- The model Skeloton is based on the paper 'Neural Collaborative Filtering', https://arxiv.org/pdf/1708.05031.pdf


# Define GMF model 
class GMF(nn.Module):
    def __init__(self, n_userIds, n_movieIds, dims=10):
        super().__init__()
        self.User_Embedding = nn.Embedding(num_embeddings = n_userIds, embedding_dim = dims)
        self.Movie_Embedding = nn.Embedding(num_embeddings = n_movieIds, embedding_dim = dims)
        
        self.linear_relu_stack = nn.Sequential(
        nn.Linear(dims, 1),
        nn.Sigmoid(),
        )

    def forward(self, u, m):
        # Get user and movie embedding
        u_embed = self.User_Embedding(u)
        m_embed = self.User_Embedding(m)
        
        # Calculate the element-wise product of the input embeddings
        x = torch.mul(u_embed, m_embed)
        # Feed it to the pony ;) -> MLP
        out = self.linear_relu_stack(x)
        return out
    
# Set up model
model = GMF(n_userIds, n_movieIds).to(device)
print(model)

In [ ]:
# Dataset for the model building
class MovieDataset(Dataset):
    def __init__(self, df, n_u, n_m):
        self.df = df
        self.n_u = n_u
        self.n_m = n_m

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        x = self.df.iloc[idx,:].values
        u, m, r = x[0], x[1], x[2]
        r_scaled = (r-1)/9
        return u, m, torch.tensor(r_scaled).float()